## API AlphaVantage - Intradiario

#### Importar las librerías necesarias

In [42]:
import requests
import json
import pandas as pd
import psycopg2
from decouple import config  # python-decouple

#### Leer la contraseña y el token desde la variable de entorno

In [43]:
password = config('DATABASE_PASSWORD')
token = config('ALPHAVANTAGE_TOKEN')

#### Definir la función para obtener datos de AlphaVantage

In [44]:
def getIntra(symbol, interval, size): 
    function = 'TIME_SERIES_INTRADAY'
    url = 'https://www.alphavantage.co/query'
    parametros = {'function': function, 'symbol': symbol, 'interval': interval, 'outputsize': size, 'datatype': json, 'apikey': token}
    
    r = requests.get(url, params=parametros)
    data = r.json()['Time Series ('+interval+')']
    dataDF = pd.DataFrame.from_dict(data, orient='index') # Convertir a DataFrame
    dataDF = dataDF.astype('float')
    dataDF.index.name = 'Fecha'
    dataDF.columns = ['Apertura', 'Maximo', 'Minimo', 'Cierre', 'Volumen']
    dataDF = dataDF.sort_values('Fecha', ascending=True)
    dataDF.index = pd.to_datetime(dataDF.index)
    return dataDF

acciones = ['AMZN', 'MELI', 'AAPL', 'GOOGL', 'MSFT'] # Lista de símbolos de acciones que se van a consultar


#### Crear e imprimir el Dataframe

In [45]:
combined_data = pd.DataFrame() # Inicializar un DataFrame vacío para combinar los datos

# Obtener y combinar los datos de las acciones en el DataFrame
for accion in acciones:
    data = getIntra(symbol=accion, interval='5min', size='compact')
    data['Simbolo'] = accion # Agregar una columna para el símbolo de la acción
    combined_data = pd.concat([combined_data, data])

combined_data = combined_data.round(3) # Redondear los valores a tres decimales

print(combined_data) # Imprimir el DataFrame combinado

                     Apertura  Maximo   Minimo   Cierre   Volumen Simbolo
Fecha                                                                    
2023-11-06 11:05:00    140.14  140.32  140.121  140.293  313084.0    AMZN
2023-11-06 11:10:00    140.29  140.29  139.970  140.080  358347.0    AMZN
2023-11-06 11:15:00    140.07  140.31  140.040  140.310  332828.0    AMZN
2023-11-06 11:20:00    140.31  140.37  140.045  140.110  346792.0    AMZN
2023-11-06 11:25:00    140.11  140.16  139.970  140.000  293708.0    AMZN
...                       ...     ...      ...      ...       ...     ...
2023-11-06 19:00:00    356.22  356.40  356.080  356.380    2083.0    MSFT
2023-11-06 19:05:00    356.38  356.40  356.220  356.240     279.0    MSFT
2023-11-06 19:10:00    356.40  356.40  356.220  356.350     407.0    MSFT
2023-11-06 19:15:00    356.31  356.48  356.310  356.340    1166.0    MSFT
2023-11-06 19:20:00    356.47  356.47  356.400  356.400     102.0    MSFT

[500 rows x 6 columns]


#### Conectarse a la base de datos PostgreSQL

In [46]:
try:
    conn = psycopg2.connect(host="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com", port=5439, database="data-engineer-database", user="hvzambrana_coderhouse", password=password)
    print("Conexión a la base de datos exitosa")
except Exception as e:
    print("Falló al crear la conexión")
    print(e)

Conexión a la base de datos exitosa


#### Crear la tabla (si no existe)

In [47]:
try:
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS hvzambrana_coderhouse.alphavantage (
        simbolo VARCHAR(10) DISTKEY NOT NULL,
        fecha TIMESTAMP SORTKEY NOT NULL,
        apertura FLOAT(15) NULL,
        maximo FLOAT(15) NULL,
        minimo FLOAT(15) NULL,
        cierre FLOAT(15) NULL,
        volumen FLOAT(15) NULL
    );""")

    conn.commit()

except Exception as e:
    print("Falló al crear la tabla")
    print(e)

#### Insertar los datos en la tabla

In [48]:
try:
    cur = conn.cursor()
    
    print("Número de registros a insertar:", len(combined_data)) # Agregar la cantidad de registros a insertar
    
    for index, row in combined_data.iterrows():
        try:
            cur.execute("""
            INSERT INTO hvzambrana_coderhouse.alphavantage (simbolo, fecha, apertura, maximo, minimo, cierre, volumen)
            VALUES (%s, %s, %s, %s, %s, %s, %s);""",
            (row['Simbolo'], index, row['Apertura'], row['Maximo'], row['Minimo'], row['Cierre'], row['Volumen']))

        except psycopg2.IntegrityError as e:
            print(f"Registro duplicado para fecha {index}. No se ha insertado.")

    conn.commit()
    print("Datos insertados con éxito")

except Exception as e:
    print("Falló al insertar datos")
    print(e)

Número de registros a insertar: 500
Datos insertados con éxito


#### Cerrar la conexión a la base de datos

In [49]:
conn.close()